### Imports and GUI Code

In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

# Output widgets for separate displays
info_output_initial = widgets.Output()
info_output_last_successful = widgets.Output()

# Additional output widget for the CSV overview
csv_overview_output = widgets.Output()

def display_csv_overview(filename, quality_preserved_only, ignore_short_texts):
    directory = "./eval/results/"
    csv_filepath = os.path.join(directory, filename)
    with csv_overview_output:
        csv_overview_output.clear_output()
        if not os.path.exists(csv_filepath):
            display(Markdown(f"**File `{filename}` not found. Please check the filename and try again.**"))
            return

        df = pd.read_csv(csv_filepath)

        if quality_preserved_only:
            df = df[df['quality_preserved'] == True]
        if ignore_short_texts:
            df = df[df['mutated_text_len'] >= 0.95 * df['current_text_len']]

        display_info = [
            f"**File name:** {filename}",
            f"**Starting z-score:** {df.iloc[0]['watermark_score']}",
            f"**Total perturbation attempts:** {len(df) - 1}",
            f"**Short length count:** {df[df['mutated_text_len'] < 0.95 * df['current_text_len']].shape[0]}",
            f"**Quality preserved perturbations:** {df['quality_preserved'].sum()}",
            f"**Attack successful:** {not df.iloc[-1]['watermark_detected']}",
        ]
        
        display(Markdown('\n\n'.join(display_info)))

# Widgets for filtering
quality_preserved_checkbox = widgets.Checkbox(value=False, description='Quality Preserved Only', disabled=False)
ignore_short_texts_checkbox = widgets.Checkbox(value=False, description='Ignore Short Texts', disabled=False)

def on_filter_change(change):
    display_csv_overview(filename_input.value, quality_preserved_checkbox.value, ignore_short_texts_checkbox.value)
    
# Observing changes in the checkboxes
quality_preserved_checkbox.observe(on_filter_change, names='value')
ignore_short_texts_checkbox.observe(on_filter_change, names='value')

def highlight_new_words(initial_text, perturbed_text):
    """Highlight new words in the perturbed text that are not in the initial text."""
    initial_words = set(initial_text.split())
    perturbed_words = perturbed_text.split()
    highlighted_text = ' '.join([f"<span style='background-color: #ccffcc'>{word}</span>" if word not in initial_words else word for word in perturbed_words])
    return highlighted_text

def display_with_initial_version(filename, perturbation_idx):
    directory = "./eval/results/"
    csv_filepath = os.path.join(directory, filename)
    with info_output_initial:
        info_output_initial.clear_output()
        if not os.path.exists(csv_filepath):
            display(Markdown(f"**File `{filename}` not found. Please check the filename and try again.**"))
            return

        df = pd.read_csv(csv_filepath)

        if perturbation_idx.isdigit():
            perturbation_idx = int(perturbation_idx)
            if 0 <= perturbation_idx < len(df):
                initial_text = df.iloc[0]['mutated_text']
                selected_perturbed_text = df.iloc[perturbation_idx]['mutated_text']
                current_diff_highlighted = highlight_new_words(initial_text, selected_perturbed_text)

                display_info = [
                    f"**Perturbation Index:** {perturbation_idx}",
                    f"**Initial Version:** {initial_text}",
                    f"**Selected Perturbed Version:** {current_diff_highlighted}",
                ]
            else:
                display_info = ["**Perturbed version index out of range. Please enter a valid index.**"]
        else:
            display_info = ["**Please enter a valid perturbation index.**"]

        display(Markdown('\n\n'.join(display_info)))

def display_with_last_successful(filename, perturbation_idx):
    directory = "./eval/results/"
    csv_filepath = os.path.join(directory, filename)
    with info_output_last_successful:
        info_output_last_successful.clear_output()
        if not os.path.exists(csv_filepath):
            display(Markdown(f"**File `{filename}` not found. Please check the filename and try again.**"))
            return

        df = pd.read_csv(csv_filepath)

        if perturbation_idx.isdigit():
            perturbation_idx = int(perturbation_idx)
            last_success = df[(df.index < perturbation_idx) & (df['quality_preserved'] == True)].tail(1)
            initial_text = df.iloc[0]['mutated_text']

            if not last_success.empty:
                last_success_text = last_success.iloc[0]['mutated_text']
                selected_perturbed_text = df.iloc[perturbation_idx]['mutated_text']
                last_success_highlighted = highlight_new_words(initial_text, last_success_text)
                current_diff_highlighted = highlight_new_words(last_success_text, selected_perturbed_text)

                display_info = [
                    f"**Perturbation Index:** {perturbation_idx}",
                    f"**Last Successful Perturbed Version:** {last_success_highlighted}",
                    f"**Selected Perturbed Version:** {current_diff_highlighted}",
                ]
            else:
                display_info = ["**No successful perturbation found before this index.**"]
        else:
            display_info = ["**Please enter a valid perturbation index.**"]

        display(Markdown('\n\n'.join(display_info)))

# Widgets for input
filename_input = widgets.Text(value='', placeholder='Enter CSV filename (e.g., completion_1.csv)', description='Filename:', disabled=False, style={'description_width': 'initial'})
perturbation_idx_input = widgets.Text(value='', placeholder='Enter perturbation index', description='Perturbation Index:', disabled=False, style={'description_width': 'initial'})

def on_input_change(change):
    display_with_initial_version(filename_input.value, perturbation_idx_input.value)
    display_with_last_successful(filename_input.value, perturbation_idx_input.value)


### The Good Stuff

In [2]:

filename_input.observe(on_input_change, names='value')
perturbation_idx_input.observe(on_input_change, names='value')

# Display input widgets and initially empty output widgets
display(filename_input, perturbation_idx_input)


Text(value='', description='Filename:', placeholder='Enter CSV filename (e.g., completion_1.csv)', style=TextS…

Text(value='', description='Perturbation Index:', placeholder='Enter perturbation index', style=TextStyle(desc…

In [3]:

# Display the widgets and the initially empty CSV overview output widget
display(quality_preserved_checkbox, ignore_short_texts_checkbox)
display(csv_overview_output)

Checkbox(value=False, description='Quality Preserved Only')

Checkbox(value=False, description='Ignore Short Texts')

Output()

In [4]:
display(info_output_initial)

Output()

In [5]:
display(info_output_last_successful)

Output()